In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
dataset_dir = '/content/drive/MyDrive/IndoorOutdoorDataset'

In [24]:
img_width, img_height = 224, 224
batch_size = 32
epochs = 100

In [25]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='training',
)

Found 800 files belonging to 2 classes.
Using 640 files for training.


In [26]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='validation',
)

Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [27]:
model = models.Sequential()
model.add(ResNet50V2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(264, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_3 (Dense)             (None, 264)               26493192  
                                                                 
 dropout_2 (Dropout)         (None, 264)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               33920     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                

In [29]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [30]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds
)

Epoch 1/100
20/20 [==============================] - 35s 401ms/step - loss: 1.2092 - accuracy: 0.7937 - val_loss: 402.5645 - val_accuracy: 0.5750
Epoch 2/100
20/20 [==============================] - 8s 341ms/step - loss: 0.6456 - accuracy: 0.8797 - val_loss: 396.5883 - val_accuracy: 0.5063
Epoch 3/100
20/20 [==============================] - 8s 358ms/step - loss: 0.5608 - accuracy: 0.8734 - val_loss: 102.3996 - val_accuracy: 0.4938
Epoch 4/100
20/20 [==============================] - 8s 343ms/step - loss: 0.4065 - accuracy: 0.8938 - val_loss: 1163.1644 - val_accuracy: 0.5000
Epoch 5/100
20/20 [==============================] - 8s 352ms/step - loss: 0.3701 - accuracy: 0.8938 - val_loss: 677.3427 - val_accuracy: 0.4625
Epoch 6/100
20/20 [==============================] - 8s 358ms/step - loss: 0.4001 - accuracy: 0.8875 - val_loss: 3200.8093 - val_accuracy: 0.5063
Epoch 7/100
20/20 [==============================] - 9s 353ms/step - loss: 0.2925 - accuracy: 0.8938 - val_loss: 85.7660 - val_

In [31]:
model.save('/content/drive/MyDrive/resnet50v2dropout_indoor_outdoor_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
model.evaluate(val_ds)

5/5 [==============================] - 1s 98ms/step - loss: 0.3259 - accuracy: 0.8687


[0.3258998990058899, 0.8687499761581421]

In [33]:
def process_directory(model, prediction_dir):
    for img in os.listdir(prediction_dir):
        try:
            img_path = os.path.join(prediction_dir, img)

            img = image.load_img(img_path, target_size=(img_width, img_height))
            img_array = image.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0)

            img_array /= 255.0

            predictions = model.predict(img_array)
            predicted_class = 'outdoor' if predictions[0] >= 0.5 else 'indoor'
            probability = predictions[0]

            print(f'{img} - Predicted Class: {predicted_class}, Probability: {probability}')

        except Exception as e:
            print(f'Exception processing {img}: {str(e)}')

In [34]:
class_labels = None

In [ ]:
outdoor_prediction_dir = '/content/drive/MyDrive/IndoorOutdoorDataset/indoor'
process_directory(model, outdoor_prediction_dir)

In [ ]:
outdoor_prediction_dir = '/content/drive/MyDrive/IndoorOutdoorDataset/outdoor'
process_directory(model, outdoor_prediction_dir)

In [ ]:
predicted_prediction_dir = '/content/drive/MyDrive/predicted'
process_directory(model, predicted_prediction_dir)